In [1]:
import pandas as pd

In [2]:
from pathlib import Path

In [3]:
ls ~/projects/gsa_coe/copybooks/PLAS.MISSING.CPYBOOKS.cpy/

PFCSI01N  PHYG0P    PHYG0S    PHYG25    PHYG3C
PHYG0O    PHYG0R    PHYG1I    PHYG3B


In [67]:
!cat ~/projects/gsa_coe/copybooks/PLAS.MISSING.CPYBOOKS.cpy/PHYG3B

014500 01  PARM-RECORD.                                                         
014600     COPY PHYG3B.                                                         
000100************************************************************              
000200* MEMBER:  PHYG3B (FDPARMIN)                               *              
000300* PURPOSE: DEFINES INPUT PARAMETERS FOR EIS PROGRAM RUNS.  *              
000400* CREATED: JANUARY 1998                                    *              
000500************************************************************              
000600* PARM RECORD LAYOUT IS:                                                  
000700*                                                                         
000800*    1         2         3         5         6         7         8        
000900* 7890123456789012345678901234567890123456789012345678901234567890        
001000*-----------------------------------------------------------------        
001100*00000000 00000000 000

# OCCURS goes before PIC clause

In [27]:
!grep -n1 "OCCURS" PLAS.MISSING.CPYBOOKS.cpy/*

PLAS.MISSING.CPYBOOKS.cpy/PFCSI01N-10-               09 programCode3-num              PIC S9(9) COMP-5 SYNC.          
PLAS.MISSING.CPYBOOKS.cpy/PFCSI01N:11:               09 programCode2 OCCURS 5.                                        
PLAS.MISSING.CPYBOOKS.cpy/PFCSI01N-12-                 12 programCode-num             PIC S9(9) COMP-5 SYNC.          
--
PLAS.MISSING.CPYBOOKS.cpy/PFCSI01N-33-               09 borrower2-num                 PIC S9(9) COMP-5 SYNC.          
PLAS.MISSING.CPYBOOKS.cpy/PFCSI01N:34:               09 borrower OCCURS 5.                                            
PLAS.MISSING.CPYBOOKS.cpy/PFCSI01N-35-                 12 borrowerId                  PIC X(10).                      
--
PLAS.MISSING.CPYBOOKS.cpy/PFCSI01N-41-               09 state2-num                    PIC S9(9) COMP-5 SYNC.          
PLAS.MISSING.CPYBOOKS.cpy/PFCSI01N:42:               09 state OCCURS 5.                                               
PLAS.MISSING.CPYBOOKS.cpy/PFCSI01N-43-    

# REDEFINES goes before PIC clause

In [2]:
!grep -n1 "REDEFINES" PLAS.MISSING.CPYBOOKS.cpy/*

PLAS.MISSING.CPYBOOKS.cpy/PHYG0S-54-004300      10 F-MO-TABLE-01                          PIC    S9(07).            
PLAS.MISSING.CPYBOOKS.cpy/PHYG0S:55:004400      10 F-MO-TABLE-01P REDEFINES F-MO-TABLE-01 PIC  -----99.             
PLAS.MISSING.CPYBOOKS.cpy/PHYG0S-56-                                                                                
--
PLAS.MISSING.CPYBOOKS.cpy/PHYG0S-61-004900*     10 F-MO-TABLE-02                          PIC    S9(05).            
PLAS.MISSING.CPYBOOKS.cpy/PHYG0S:62:005000*     10 F-MO-TABLE-02P REDEFINES F-MO-TABLE-02 PIC    ---99.             
PLAS.MISSING.CPYBOOKS.cpy/PHYG0S-63-005100*                                                                         


# VALUE is either in lieu of or after PIC clause

In [3]:
!grep -n1 "VALUE" PLAS.MISSING.CPYBOOKS.cpy/*

PLAS.MISSING.CPYBOOKS.cpy/PHYG0O-50-2C        005000         10  RECORD-TYPE-FLAG               PIC X(01).          
PLAS.MISSING.CPYBOOKS.cpy/PHYG0O:51:3C        005100             88  MASTER-RECORD-TYPE         VALUE 'M'.          
PLAS.MISSING.CPYBOOKS.cpy/PHYG0O:52:4C        005200             88  DETAIL-RECORD-TYPE         VALUE 'D'.          
PLAS.MISSING.CPYBOOKS.cpy/PHYG0O-53-5C        005300         10  FLAG-2                         PIC X(01).          
--
PLAS.MISSING.CPYBOOKS.cpy/PHYG0P-39-4C        006400         10  RECORD-TYPE                    PIC 9(01).          
PLAS.MISSING.CPYBOOKS.cpy/PHYG0P:40:5C        006500             88  COUNTY-RECORD              VALUE  1.           
PLAS.MISSING.CPYBOOKS.cpy/PHYG0P:41:6C        006600             88  HEADQUARTERS-RECORD        VALUE  2.           
PLAS.MISSING.CPYBOOKS.cpy/PHYG0P:42:7C        006700             88  DISTRICT-RECORD            VALUE  3.           
PLAS.MISSING.CPYBOOKS.cpy/PHYG0P:43:8C        006800         

# Read in copybooks and normalize their form across the different files

In [4]:
copybook_dir = Path( '/Users/ccoletta/projects/gsa_coe/copybooks/PLAS.MISSING.CPYBOOKS.cpy/' )

In [5]:
#pd.read_csv?

In [6]:
dfs = {}
for p in copybook_dir.glob( '*' ):
    name = p.stem[:6]
    try:
        df = pd.read_csv( p, sep='\t', header=None )
    except UnicodeDecodeError as e:
        continue
    dfs[ name ] = pd.Series( df[0], name=name )

In [7]:
len( dfs )

8

In [8]:
clip_these =['PHYG0O', 'PHYG0P', 'PHYG1I',]

In [9]:
len( "1         " )

10

In [10]:
len( "4C        " )

10

In [11]:
len( "7C*       " )

10

In [12]:
#dfs

In [13]:
for name, df in dfs.items():
    if name not in clip_these:
        continue
    dfs[ name ] = df.apply( lambda val: val[10:] )

In [14]:
allrows = pd.DataFrame( dfs )

In [15]:
allrows = allrows.melt( var_name='program', value_name='raw_line' ).dropna()

In [16]:
allrows

,program,raw_line
0,PHYG0R,002790*01 TOTAL-RECORD. ...
1,PHYG0R,002800 COPY PHYG0R. ...
2,PHYG0R,000100****************************************...
3,PHYG0R,000200* MEMBER: PHYG0R (WSMASTER) ...
4,PHYG0R,000300* PURPOSE: FDMASTER STANDARD EIS FILE FO...
...,...,...
1119,PHYG1I,004000 10 TABLE-START-DY PIC X(2...
1120,PHYG1I,004100 05 BLANK-13 PIC X(1...
1121,PHYG1I,004200 05 ROLL-FLAG PIC X(1...
1122,PHYG1I,004300 05 FILLER PIC X(2...


In [17]:
allrows['no_decl_num'] = allrows['raw_line'].str.slice( start=6 )

In [18]:
no_comment_rows = allrows[ ~allrows['no_decl_num'].str.startswith( '*' ) ].copy()

In [19]:
len( no_comment_rows ) 

537

In [20]:
no_comment_rows['rstripped'] = no_comment_rows['no_decl_num'].str.rstrip()

In [21]:
no_comment_rows['rstripped'].str.len()

1       17
8        0
9       24
10      25
11      54
        ..
1119    42
1120    42
1121    42
1122    43
1123     0
Name: rstripped, Length: 537, dtype: int64

In [22]:
no_blanks = no_comment_rows[ no_comment_rows['rstripped'].str.len() != 0 ].copy()

In [23]:
len( no_blanks) 

494

In [24]:
no_copys = no_blanks[ ~no_blanks['rstripped'].str.contains( 'COPY' ) ].copy()

In [25]:
len( no_copys )

486

In [26]:
pd.set_option( 'display.max_rows', None )
pd.set_option( 'display.max_colwidth', None )

# Generate statistics about continuation of lines

In [27]:
no_copys['continued'] = ~(no_copys['rstripped'].str.slice( start=-1 ) == '.')

In [28]:
continuation_lines = [ _+1 for _ in list( no_copys[ no_copys['continued'] ].index ) ]

In [29]:
no_copys[ no_copys['continued'] ].index

Int64Index([28, 29, 67, 349, 822, 823, 862, 1103], dtype='int64')

In [30]:
continuation_lines

[29, 30, 68, 350, 823, 824, 863, 1104]

In [31]:
no_copys['continuation'] = False

In [32]:
no_copys.loc[ continuation_lines, 'continuation' ] = True

In [33]:
#no_copys['rstripped'].values

In [34]:
stats_df = no_copys['rstripped'].str.extract( r'(\s+)(\S+)' )

In [35]:
stats_df.columns = ['leading_whitespace', 'first_token' ]

In [36]:
stats_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 486 entries, 9 to 1122
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   leading_whitespace  486 non-null    object
 1   first_token         486 non-null    object
dtypes: object(2)
memory usage: 27.6+ KB


In [37]:
stats_df['leading_whitespace'] = stats_df['leading_whitespace'].str.len()

In [38]:
set( stats_df.index ) == set( no_copys.index )

True

In [39]:
stats_df.shape

(486, 2)

In [40]:
no_copys.shape

(486, 6)

In [41]:
copybook_data = pd.concat( ( stats_df, no_copys ), axis=1 )

In [42]:
#copybook_data

In [43]:
copybook_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 486 entries, 9 to 1122
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   leading_whitespace  486 non-null    int64 
 1   first_token         486 non-null    object
 2   program             486 non-null    object
 3   raw_line            486 non-null    object
 4   no_decl_num         486 non-null    object
 5   rstripped           486 non-null    object
 6   continued           486 non-null    bool  
 7   continuation        486 non-null    bool  
dtypes: bool(2), int64(1), object(5)
memory usage: 43.7+ KB


In [44]:
non_continued = copybook_data[ copybook_data['continuation'] == False ]

In [45]:
non_continued.shape

(478, 8)

In [46]:
non_continued['program'].value_counts()

PFCSI0    152
PHYG25     76
PHYG0P     59
PHYG0R     51
PHYG0O     49
PHYG1I     34
PHYG3C     29
PHYG0S     28
Name: program, dtype: int64

In [47]:
non_continued['leading_whitespace'].value_counts()

9     196
5     118
11     92
13     49
1      12
7       7
6       2
2       1
17      1
Name: leading_whitespace, dtype: int64

In [48]:
non_continued['first_token'].value_counts()

10    146
05    117
12     89
09     60
15     37
88     14
01     13
03      1
06      1
Name: first_token, dtype: int64

In [49]:
copybook_data.loc[ non_continued.index, 'PIC_index'] = non_continued['raw_line'].apply( lambda line: line.index('PIC') if 'PIC' in line else -1 )

In [52]:
copybook_data.groupby( 'program' )['PIC_index'].value_counts()

program  PIC_index
PFCSI0    48.0        145
         -1.0           7
PHYG0O    50.0         37
         -1.0          11
          55.0          1
PHYG0P    50.0         37
         -1.0          22
PHYG0R    50.0         37
         -1.0          14
PHYG0S    41.0         16
         -1.0           9
          54.0          2
          32.0          1
PHYG1I    39.0         30
         -1.0           4
PHYG25    45.0         67
         -1.0           9
PHYG3C    50.0         22
         -1.0           7
Name: PIC_index, dtype: int64

In [53]:
copybook_data

,leading_whitespace,first_token,program,raw_line,no_decl_num,rstripped,continued,continuation,PIC_index,new_leading_whitespace
9,2,01,PHYG0R,000800 01 GRAND-TOTAL-RECORD.,01 GRAND-TOTAL-RECORD.,01 GRAND-TOTAL-RECORD.,False,False,-1.0,1.0
10,5,05,PHYG0R,000900 05 GRAND-SORT-KEYS.,05 GRAND-SORT-KEYS.,05 GRAND-SORT-KEYS.,False,False,-1.0,5.0
11,9,10,PHYG0R,001000 10 GRAND-COUNTRY PIC X(03).,10 GRAND-COUNTRY PIC X(03).,10 GRAND-COUNTRY PIC X(03).,False,False,50.0,9.0
12,9,10,PHYG0R,001100 10 GRAND-STATE PIC X(18).,10 GRAND-STATE PIC X(18).,10 GRAND-STATE PIC X(18).,False,False,50.0,9.0
13,9,10,PHYG0R,001200 10 GRAND-PROGRAM-NO PIC 9(03).,10 GRAND-PROGRAM-NO PIC 9(03).,10 GRAND-PROGRAM-NO PIC 9(03).,False,False,50.0,9.0
14,9,10,PHYG0R,001300 10 GRAND-PROGRAM-NAME PIC X(18).,10 GRAND-PROGRAM-NAME PIC X(18).,10 GRAND-PROGRAM-NAME PIC X(18).,False,False,50.0,9.0
15,9,10,PHYG0R,001400 10 GRAND-DISTRICT PIC X(18).,10 GRAND-DISTRICT PIC X(18).,10 GRAND-DISTRICT PIC X(18).,False,False,50.0,9.0
16,9,10,PHYG0R,001500 10 GRAND-HEADQTRS PIC X(18).,10 GRAND-HEADQTRS PIC X(18).,10 GRAND-HEADQTRS PIC X(18).,False,False,50.0,9.0
17,9,10,PHYG0R,001600 10 GRAND-COUNTY PIC X(18).,10 GRAND-COUNTY PIC X(18).,10 GRAND-COUNTY PIC X(18).,False,False,50.0,9.0
19,5,05,PHYG0R,001800 05 GRAND-KEY-FIELDS.,05 GRAND-KEY-FIELDS.,05 GRAND-KEY-FIELDS.,False,False,-1.0,5.0


# Generate statistics about indentation

In [50]:
copybook_data.loc[ non_continued.index, 'new_leading_whitespace'] = 1 + ( non_continued['first_token'].astype( int ) // 4 )*4

In [50]:
#copybook_data[ ['leading_whitespace','new_leading_whitespace'] ]

In [51]:
metadata = pd.read_excel( '2022-12-15_IDMS_table_descriptions.xlsx', index_col=0 )

In [52]:
metadata.index.name = "table_index"

In [53]:
#wanted_subsystem = "DISCREPANCY PROCESSING AND ACCOUNT INFORMATION INQUIRY"

In [54]:
#metadata[ metadata[ wanted_subsystem ] == 1 ].copy()

In [55]:
data = pd.read_excel( "2022-12-08_PLAS_IDMS_data_structure_WITH_VALID_VALUES.xlsx", index_col=0 )

In [56]:
data.shape

(89288, 18)

In [57]:
data['indent_number'].value_counts()

5     53625
10    24046
15     6156
88     2718
3      1375
20      906
4       151
2       146
25      112
7        49
9         4
Name: indent_number, dtype: int64

In [58]:
from collections import Counter

In [59]:
data.head()

,table_name,table_index,table_vers,table_created_date,table_updated_date,field_name,data_type,indent_number,indent_space_count,declaration_step,end,raw_field_name,BLANK ON,INDEXED BY,OCCURS,OLQ,REDEFINES,VALUE
0,ABSDOMN,0,1,1987-09-29,1987-10-15,ST-CTY-CDE-FMHA,NaN,5,3,100,DISPLAY,ST-CTY-CDE-FMHA-4541,NaN,NaN,NaN,NaN,NaN,NaN
1,ABSDOMN,0,1,1987-09-29,1987-10-15,ST-CDE-FMHA,9(2),10,4,200,DISPLAY,ST-CDE-FMHA-4541,NaN,NaN,NaN,NaN,NaN,NaN
2,ABSDOMN,0,1,1987-09-29,1987-10-15,CTY-CDE-FMHA,9(3),10,4,300,DISPLAY,CTY-CDE-FMHA-4541,NaN,NaN,NaN,NaN,NaN,NaN
3,ABSDOMN,0,1,1987-09-29,1987-10-15,NME-CTY,X(20),5,3,400,DISPLAY,NME-CTY-4541,NaN,NaN,NaN,NaN,NaN,NaN
4,ABSDOMN,0,1,1987-09-29,1987-10-15,LST-UPDTE-DTE,9(06),5,3,500,DISPLAY,LST-UPDTE-DTE-4541,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
data[ ['indent_number', 'indent_space_count'] ].value_counts()

indent_number  indent_space_count
5              3                     53625
10             4                     24046
15             5                      6156
88             7                      2718
3              3                      1375
20             6                       906
4              3                       151
2              3                       146
25             7                       112
7              4                        49
9              4                         4
dtype: int64

In [61]:
data[ ['table_index', 'declaration_step'] ].values

array([[   0,  100],
       [   0,  200],
       [   0,  300],
       ...,
       [3660, 5300],
       [3660, 5400],
       [3660, 5500]])

In [62]:
data.head( 30 )

,table_name,table_index,table_vers,table_created_date,table_updated_date,field_name,data_type,indent_number,indent_space_count,declaration_step,end,raw_field_name,BLANK ON,INDEXED BY,OCCURS,OLQ,REDEFINES,VALUE
0,ABSDOMN,0,1,1987-09-29,1987-10-15,ST-CTY-CDE-FMHA,NaN,5,3,100,DISPLAY,ST-CTY-CDE-FMHA-4541,NaN,NaN,NaN,NaN,NaN,NaN
1,ABSDOMN,0,1,1987-09-29,1987-10-15,ST-CDE-FMHA,9(2),10,4,200,DISPLAY,ST-CDE-FMHA-4541,NaN,NaN,NaN,NaN,NaN,NaN
2,ABSDOMN,0,1,1987-09-29,1987-10-15,CTY-CDE-FMHA,9(3),10,4,300,DISPLAY,CTY-CDE-FMHA-4541,NaN,NaN,NaN,NaN,NaN,NaN
3,ABSDOMN,0,1,1987-09-29,1987-10-15,NME-CTY,X(20),5,3,400,DISPLAY,NME-CTY-4541,NaN,NaN,NaN,NaN,NaN,NaN
4,ABSDOMN,0,1,1987-09-29,1987-10-15,LST-UPDTE-DTE,9(06),5,3,500,DISPLAY,LST-UPDTE-DTE-4541,NaN,NaN,NaN,NaN,NaN,NaN
5,ABSDOMN,0,1,1987-09-29,1987-10-15,FILLER,X,5,3,600,DISPLAY,FILLER,NaN,NaN,NaN,NaN,NaN,NaN
6,ACCRLINK,1,1,1984-02-21,1998-03-09,RQST-AREA-MFH,NaN,5,3,100,DISPLAY,W019-RQST-AREA-MFH,NaN,NaN,NaN,NaN,NaN,NaN
7,ACCRLINK,1,1,1984-02-21,1998-03-09,RQST-INT-ON-LN-MFH,X(1),10,4,200,DISPLAY,W019-RQST-INT-ON-LN-MFH,NaN,NaN,NaN,NaN,NaN,NaN
8,ACCRLINK,1,1,1984-02-21,1998-03-09,RQST-INT-CST-ITEM-MFH,X(1),10,4,300,DISPLAY,W019-RQST-INT-CST-ITEM-MFH,NaN,NaN,NaN,NaN,NaN,NaN
9,ACCRLINK,1,1,1984-02-21,1998-03-09,RQST-INT-AUDIT-RCVB-MFH,X(1),10,4,400,DISPLAY,W019-RQST-INT-AUDIT-RCVB-MFH,NaN,NaN,NaN,NaN,NaN,NaN
